## This notebook explores 21 records that were removed from the original dataset because ##the REMARKS field was either duplicate or NA
### The output of this notebook is:
#### 1. 5 Lat/Long calculated for FAA Airport Identifier
#### 2. 1  Runway record but with no IDENT  (Sent to Lex)
#### 3. 7 NA remarks (sent to Lex)
#### 4. 8 remaining duplicates records (Sent to Lex)

In [2]:
# Import libraries
import pandas as pd
import numpy as np
import geopy
from geopy.distance import geodesic
import re

In [3]:
# Read in csv file
nonstd = pd.read_csv('C:/Users/Olga/Desktop/GMU/DAEN690/LEGO/Non-std-Duplicates&NA.csv')
nonstd

,dateonly,timeonly,DATE,CALLSIGN,POD,PRIMARYCODE,SECONDARYCODES,REPORTINGFACILITY,REMARKS,MORID.CEDAR,...,NEARESTAIRPORT.CEDAR,AIRSPACECLASS.CEDAR,ACLOCATION.CEDAR,ACALTITUDE.CEDAR,ACHEADING.CEDAR,RELATIVECLOCKPOSITION.CEDAR,UASLONG.CEDAR,UASLAT.CEDAR,SUMMARY.CEDAR,QAFINDINGS.CEDAR
0,4/19/2018,6:17,2018-04-19T06:17Z,JBU454,DEN,UAS,NaN,N90,Reported observing a silver UAS operating at 4...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6/3/2018,2:11,2018-06-03T02:11Z,LOF4785,DEN,UAS,NaN,D01,"PIC reported a white UAS SW bound at 8,000 fee...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6/6/2018,2:10,2018-06-06T02:10Z,SWA1779,DEN,UAS,NaN,D01,"PIC reported a white UAS while N bound at 10,2...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,7/3/2018,19:06,2018-07-03T19:06Z,UAL641,DEN,UAS,NaN,A90,PIC observed a blue and white UAS with orange ...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,7/8/2018,5:08,2018-07-08T05:08Z,ASA748,DEN,UAS,NaN,STL,PIC reported a small black UAS while on short ...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,8/11/2018,0:34,2018-08-11T00:34Z,R51263,DEN,UAS,NaN,DHN,Pilot reported a 4 foot silver UAS orbiting at...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,10/11/2018,3:36,2018-10-11T03:36Z,AAL2286,DEN,UAS,NaN,N90,PIC reported 2 UAS below on right side while S...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,12/25/2018,7:17,2018-12-25T07:17Z,CPZ5708,DEN,UAS,NaN,LAX,PIC reported a UAS with green flashing lights ...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,3/17/2019,1:05,2019-03-17T01:05Z,NaN,DEN,UAS,NaN,BUR,Unreported call sign reported a UAS 5 NM E of ...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,5/24/2019,5:51,2019-05-24T05:51Z,UAL841,DEN,UAS,NaN,C90,Reported sighting a reflective UAS off of the ...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# Find NA remarks 
null = nonstd[nonstd['REMARKS'].isnull()]
null

,dateonly,timeonly,DATE,CALLSIGN,POD,PRIMARYCODE,SECONDARYCODES,REPORTINGFACILITY,REMARKS,MORID.CEDAR,...,NEARESTAIRPORT.CEDAR,AIRSPACECLASS.CEDAR,ACLOCATION.CEDAR,ACALTITUDE.CEDAR,ACHEADING.CEDAR,RELATIVECLOCKPOSITION.CEDAR,UASLONG.CEDAR,UASLAT.CEDAR,SUMMARY.CEDAR,QAFINDINGS.CEDAR
14,7/18/2021,18:10,2021-07-18T18:10Z,EJA761,ATO,UAS,NaN,VNY,NaN,6.638527e+08,...,KVNY,D,VNY300006,25.0,100.0,12.0,NaN,NaN,EJA761 inbound from the west reported an unman...,Reviewed MOR Summary for a UAS report. P Rober...
15,7/18/2021,20:30,2021-07-18T20:30Z,LEGCY11,ATO,UAS,NaN,GSO,NaN,7.509909e+08,...,KGSO,E,GSO175010,90.0,120.0,NaN,79.5702,35.5443,"LEGCY11, 2/V22 GSO overflight reported to RDU ...","Facility summary, weather and falcon bookmark ..."
16,7/20/2021,16:39,2021-07-20T16:39Z,JIA5265,ATO,UAS,NaN,DCA,NaN,2.169437e+07,...,KDCA,B,DCA180/003,7.0,360.0,NaN,NaN,NaN,"JIA5265 (CRJ7) REPORTED A SMALL, WHITE, SQUARE...","Facility summary, weather and falcon bookmark ..."
17,7/21/2021,3:46,2021-07-21T03:46Z,JIA5596,ATO,UAS,NaN,DCA,NaN,7.440653e+08,...,KDCA,B,SHORT FINAL RWY 19,200.0,190.0,NaN,NaN,NaN,"JIA5596 LANDED RWY 19 AND REPORTED A DRONE, WE...","Facility summary, weather and falcon bookmark ..."
18,7/24/2021,1:20,2021-07-24T01:20Z,NaN,ATO,UAS,NaN,DCA,NaN,1.620153e+09,...,KDCA,B,P56A,NaN,360.0,NaN,NaN,NaN,SECRET SERVICE CALLED TO REPORT UNAUTHORIZED U...,MOR reviewed and closed.
19,7/26/2021,16:08,2021-07-26T16:08Z,EDV4756,ATO,UAS,NaN,AZO,NaN,1.052150e+09,...,KAZO,D,AZO350002,43.0,350.0,1.0,NaN,NaN,"EDV4756 (climbing through 4,300') reported a b...",QA reviewed.
20,7/27/2021,20:56,2021-07-27T20:56Z,N949SC,ATO,UAS,NaN,LFT,NaN,7.757858e+08,...,KLFT,C,LFT/340/05,20.0,100.0,9.0,NaN,NaN,N949SC NOTFIED THE WEST RADAR CONTROLLER THAT...,QA reviewed.


In [4]:
# Export to csv file and sent back to Lex
null.to_csv('Non-std-7NA-Remarks.csv', index = False)

In [10]:
# Find the non null remarks
duplicate = nonstd[nonstd['REMARKS'].notnull()]
duplicate

,dateonly,timeonly,DATE,CALLSIGN,POD,PRIMARYCODE,SECONDARYCODES,REPORTINGFACILITY,REMARKS,MORID.CEDAR,...,NEARESTAIRPORT.CEDAR,AIRSPACECLASS.CEDAR,ACLOCATION.CEDAR,ACALTITUDE.CEDAR,ACHEADING.CEDAR,RELATIVECLOCKPOSITION.CEDAR,UASLONG.CEDAR,UASLAT.CEDAR,SUMMARY.CEDAR,QAFINDINGS.CEDAR
0,4/19/2018,6:17,2018-04-19T06:17Z,JBU454,DEN,UAS,NaN,N90,Reported observing a silver UAS operating at 4...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6/3/2018,2:11,2018-06-03T02:11Z,LOF4785,DEN,UAS,NaN,D01,"PIC reported a white UAS SW bound at 8,000 fee...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6/6/2018,2:10,2018-06-06T02:10Z,SWA1779,DEN,UAS,NaN,D01,"PIC reported a white UAS while N bound at 10,2...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,7/3/2018,19:06,2018-07-03T19:06Z,UAL641,DEN,UAS,NaN,A90,PIC observed a blue and white UAS with orange ...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,7/8/2018,5:08,2018-07-08T05:08Z,ASA748,DEN,UAS,NaN,STL,PIC reported a small black UAS while on short ...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,8/11/2018,0:34,2018-08-11T00:34Z,R51263,DEN,UAS,NaN,DHN,Pilot reported a 4 foot silver UAS orbiting at...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,10/11/2018,3:36,2018-10-11T03:36Z,AAL2286,DEN,UAS,NaN,N90,PIC reported 2 UAS below on right side while S...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,12/25/2018,7:17,2018-12-25T07:17Z,CPZ5708,DEN,UAS,NaN,LAX,PIC reported a UAS with green flashing lights ...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,3/17/2019,1:05,2019-03-17T01:05Z,NaN,DEN,UAS,NaN,BUR,Unreported call sign reported a UAS 5 NM E of ...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,5/24/2019,5:51,2019-05-24T05:51Z,UAL841,DEN,UAS,NaN,C90,Reported sighting a reflective UAS off of the ...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# Creating List that contains each the remarks with the headings contained in it
remark_head_dir = []
remarks = duplicate['REMARKS']

# Using regular expression for the heading/direction
headir_regex = '[0-9][0-9]*[0-9]*\s?NM* [N|S|E|W|NW|NE|SW|SE|SSE|SSW|SNE|SNW|NNE|NSE|NNE|NNW|WSW|WNW|WSE|WNE|ENE|ESE|ESW|ENW]*'

# For loop through remarks to find headings
for i in range(len(remarks)):
    head_dir = re.findall(headir_regex, str(remarks[i]))
    remark_head_dir.append(remarks[i])
    remark_head_dir.append(head_dir)
    
# Split Remarks and Heading/Directions into two seperate lists and create pandas dataframe
remark = []
head_dir = []

for i in range(0, len(remark_head_dir), 2):
    remark.append(remark_head_dir[i])
    head_dir.append(remark_head_dir[i+1])

remark_head_dir_df = pd.DataFrame()
remark_head_dir_df['REMARKS'] = remark
remark_head_dir_df['Heading_Direction'] = head_dir

remark_head_dir_df

,REMARKS,Heading_Direction
0,Reported observing a silver UAS operating at 4...,[]
1,"PIC reported a white UAS SW bound at 8,000 fee...",[7 NM NW]
2,"PIC reported a white UAS while N bound at 10,2...",[9 NM NW]
3,PIC observed a blue and white UAS with orange ...,[25 NM ]
4,PIC reported a small black UAS while on short ...,[]
5,Pilot reported a 4 foot silver UAS orbiting at...,[5 NM S]
6,PIC reported 2 UAS below on right side while S...,[7 NM N]
7,PIC reported a UAS with green flashing lights ...,[1 NM ]
8,Unreported call sign reported a UAS 5 NM E of ...,[5 NM E]
9,Reported sighting a reflective UAS off of the ...,[]


In [12]:
# Create List that contains each Remark, and the Heading/Direction information contained in each remark
remark_uas_loc = []
remarks = duplicate['REMARKS']

# Using regular expression for any heading/direction
headir_regex = '[0-9][0-9]*[0-9]*\s?NM* [N|S|E|W|NW|NE|SW|SE|SSE|SSW|SNE|SNW|NNE|NSE|NNE|NNW|WSW|WNW|WSE|WNE|ENE|ESE|ESW|ENW]*\s?of?\s?[A-Z][A-Z][A-Z][A-Z]?'

# Loop through all remarks and search for the heading/direction regex above
for i in range(len(remarks)):
    head_dir = re.findall(headir_regex, str(remarks[i]))
    remark_uas_loc.append(remarks[i])
    remark_uas_loc.append(head_dir)

# Split Remarks and Heading/Directions into two seperate lists and create pandas dataframe
remark = []
uas_loc = []

for i in range(0, len(remark_uas_loc), 2):
    remark.append(remark_uas_loc[i])
    uas_loc.append(remark_uas_loc[i+1])

remark_uas_loc_df = pd.DataFrame()
remark_uas_loc_df['REMARKS'] = remark
remark_uas_loc_df['UAS Location'] = uas_loc
remark_uas_loc_df

,REMARKS,UAS Location
0,Reported observing a silver UAS operating at 4...,[]
1,"PIC reported a white UAS SW bound at 8,000 fee...",[7 NM NW of DEN]
2,"PIC reported a white UAS while N bound at 10,2...",[9 NM NW of DEN]
3,PIC observed a blue and white UAS with orange ...,[]
4,PIC reported a small black UAS while on short ...,[]
5,Pilot reported a 4 foot silver UAS orbiting at...,[5 NM S of RWY]
6,PIC reported 2 UAS below on right side while S...,[7 NM N of LGA]
7,PIC reported a UAS with green flashing lights ...,[]
8,Unreported call sign reported a UAS 5 NM E of ...,[5 NM E of BUR]
9,Reported sighting a reflective UAS off of the ...,[]


In [13]:
uas_loc = remark_uas_loc_df['UAS Location'] 

# If the regular expressions did not hit on any location information, pass it UNKN for now
for i in range(len(remark_uas_loc_df)):
    if len(uas_loc[i]) == 0:
        uas_loc[i] = 'UNKN'

In [14]:
len(uas_loc[uas_loc != 'UNKN']) # 6 records have UAS Location

6

In [11]:
# Find the records with no UAS location and save them in csv file and sent to Lex for further inverstigation or to 
# Exceptions file
exception = remark_uas_loc_df[remark_uas_loc_df['UAS Location'] == 'UNKN']
exception = pd.merge(exception, nonstd, on='REMARKS', how='left')
exception.to_csv('Non-std-8RemainingOfDuplicates.csv', index = False)

In [15]:
# Read in airports dataset
airportsC = pd.read_csv('C:/Users/Olga/Desktop/GMU/DAEN690/LEGO/airports_cleaned.csv')

In [16]:
# Save the records with UAS location in new dataframe 
remark_uas_loc_nn = remark_uas_loc_df[remark_uas_loc_df['UAS Location'] != 'UNKN'].reset_index()
remark_uas_loc_nn

,index,REMARKS,UAS Location
0,1,"PIC reported a white UAS SW bound at 8,000 fee...",[7 NM NW of DEN]
1,2,"PIC reported a white UAS while N bound at 10,2...",[9 NM NW of DEN]
2,5,Pilot reported a 4 foot silver UAS orbiting at...,[5 NM S of RWY]
3,6,PIC reported 2 UAS below on right side while S...,[7 NM N of LGA]
4,8,Unreported call sign reported a UAS 5 NM E of ...,[5 NM E of BUR]
5,10,"PIC reported a UAS while NNE bound at 1,000 fe...",[1 NM N of HWD]


In [17]:
# Extract all IDENTs
uas_loc_nonNull = uas_loc[uas_loc != 'UNKN'].to_list()
uas_airport = []

for i in range(len(uas_loc_nonNull)):
    airport = uas_loc_nonNull[i][0].split(' ')[-1]
    
    if len(airport) <= 4:
        uas_airport.append(airport)
    else:
        trim_air = airport[-3:]
        uas_airport.append(trim_air)
        
uas_airport_df = pd.DataFrame()
uas_airport_df['IDENT'] = uas_airport
uas_airport_df

,IDENT
0,DEN
1,DEN
2,RWY
3,LGA
4,BUR
5,HWD


In [18]:
# Merge Idents with airport dataset
uas_air_loc= pd.merge(uas_airport_df, airportsC, on='IDENT', how='left')
full_loc = pd.DataFrame()
full_loc['REMARKS'] = remark_uas_loc_nn['REMARKS']
full_loc['UAS_LOC'] = remark_uas_loc_nn['UAS Location']

final_df1 = pd.concat([full_loc, uas_air_loc], axis = 1)
final_df1

,REMARKS,UAS_LOC,IDENT,NAME,LATITUDE,LONGITUDE,ICAO_ID
0,"PIC reported a white UAS SW bound at 8,000 fee...",[7 NM NW of DEN],DEN,Denver Intl,39.861667,-104.673167,KDEN
1,"PIC reported a white UAS while N bound at 10,2...",[9 NM NW of DEN],DEN,Denver Intl,39.861667,-104.673167,KDEN
2,Pilot reported a 4 foot silver UAS orbiting at...,[5 NM S of RWY],RWY,NaN,NaN,NaN,NaN
3,PIC reported 2 UAS below on right side while S...,[7 NM N of LGA],LGA,Laguardia,40.777250,-73.872611,KLGA
4,Unreported call sign reported a UAS 5 NM E of ...,[5 NM E of BUR],BUR,Bob Hope,34.200694,-118.358667,KBUR
5,"PIC reported a UAS while NNE bound at 1,000 fe...",[1 NM N of HWD],HWD,Hayward Exec,37.658929,-122.121737,KHWD


In [19]:
# Find the records that reference airports
# We have 5 records
uas_air_loc = final_df1[final_df1['NAME'].notnull()]
uas_air_loc.to_csv('5airport.csv', index = False)
uas_air_loc = pd.read_csv('C:/Users/Olga/Desktop/GMU/DAEN690/LEGO/5airport.csv')
uas_air_loc

,REMARKS,UAS_LOC,IDENT,NAME,LATITUDE,LONGITUDE,ICAO_ID
0,"PIC reported a white UAS SW bound at 8,000 fee...",['7 NM NW of DEN'],DEN,Denver Intl,39.861667,-104.673167,KDEN
1,"PIC reported a white UAS while N bound at 10,2...",['9 NM NW of DEN'],DEN,Denver Intl,39.861667,-104.673167,KDEN
2,PIC reported 2 UAS below on right side while S...,['7 NM N of LGA'],LGA,Laguardia,40.777250,-73.872611,KLGA
3,Unreported call sign reported a UAS 5 NM E of ...,['5 NM E of BUR'],BUR,Bob Hope,34.200694,-118.358667,KBUR
4,"PIC reported a UAS while NNE bound at 1,000 fe...",['1 NM N of HWD'],HWD,Hayward Exec,37.658929,-122.121737,KHWD


In [20]:
# Create Dictionary of Bearings and Respective Degrees
bearing_deg = {
    'N' : 0,
    'NNE' : 23,
    'NE' : 45,
    'ENE' : 68,
    'E' : 90,
    'ESE' : 113,
    'SE' : 135,
    'SSE' : 158,
    'S' : 180,
    'SSW' : 203,
    'SW' : 225,
    'WSW' : 248,
    'W' : 270,
    'WNW' : 293,
    'NW' : 315,
    'NNW' : 338
}

# Separate distances and bearings into two columns
uas_loc = uas_air_loc['UAS_LOC']
distances = []
bearings = []

for i in range(len(uas_loc)):
    split = uas_loc[i].split(' ')[0:3]
    
    if 'NM' in split[0]: #if there is no space between the distance and NM (i.e. 3NM)
        distance_nm = split[0][0:len(split[0]) - 2]
        bearing_abrev = split[1].replace('of', '')
        
    else: 
        distance_nm = split[0][2:len(split[0])]
        bearing_abrev = split[2].replace('of', '')
    
    distances.append(distance_nm[0:])
    bearings.append(bearing_abrev)

# Create DataFrame to store UAS Location Information
new_df = pd.DataFrame(uas_air_loc['REMARKS'])
new_df['UAS_LOC'] = uas_air_loc['UAS_LOC']
new_df['IDENT'] = uas_air_loc['IDENT']
new_df['Distance'] = distances
new_df['Bearing'] = bearings

new_df

,REMARKS,UAS_LOC,IDENT,Distance,Bearing
0,"PIC reported a white UAS SW bound at 8,000 fee...",['7 NM NW of DEN'],DEN,7,NW
1,"PIC reported a white UAS while N bound at 10,2...",['9 NM NW of DEN'],DEN,9,NW
2,PIC reported 2 UAS below on right side while S...,['7 NM N of LGA'],LGA,7,N
3,Unreported call sign reported a UAS 5 NM E of ...,['5 NM E of BUR'],BUR,5,E
4,"PIC reported a UAS while NNE bound at 1,000 fe...",['1 NM N of HWD'],HWD,1,N


In [22]:
# Convert distance from NM to KM and bearings to degrees
new_df['Distance'] = pd.to_numeric(new_df['Distance'],errors = 'coerce')
dist_kilo = []

for i in range(len(new_df)):
    distanceKilo = new_df['Distance'][i] * 1.852 # converting NM to kilometers
    dist_kilo.append(distanceKilo)

bearDegree = pd.DataFrame(new_df['Bearing'])
bearDegree = bearDegree.replace({'Bearing': bearing_deg})

new_df['Distance_Kilometers'] = dist_kilo
new_df['Bearing_Degrees'] = bearDegree
new_df['Airport_Latitude'] = uas_air_loc['LATITUDE']
new_df['Airport_Longitude'] = uas_air_loc['LONGITUDE']

new_df2 = pd.merge(new_df, airportsC, on='IDENT', how='left')
new_df2 = new_df2.drop(['Airport_Latitude', 'Airport_Longitude'], 1)
new_df2 = new_df2.rename({'LONGITUDE': 'Airport_Longitude', 'LATITUDE': 'Airport_Latitude'}, axis=1)
new_df2

,REMARKS,UAS_LOC,IDENT,Distance,Bearing,Distance_Kilometers,Bearing_Degrees,NAME,Airport_Latitude,Airport_Longitude,ICAO_ID
0,"PIC reported a white UAS SW bound at 8,000 fee...",['7 NM NW of DEN'],DEN,7,NW,12.964,315,Denver Intl,39.861667,-104.673167,KDEN
1,"PIC reported a white UAS while N bound at 10,2...",['9 NM NW of DEN'],DEN,9,NW,16.668,315,Denver Intl,39.861667,-104.673167,KDEN
2,PIC reported 2 UAS below on right side while S...,['7 NM N of LGA'],LGA,7,N,12.964,0,Laguardia,40.777250,-73.872611,KLGA
3,Unreported call sign reported a UAS 5 NM E of ...,['5 NM E of BUR'],BUR,5,E,9.260,90,Bob Hope,34.200694,-118.358667,KBUR
4,"PIC reported a UAS while NNE bound at 1,000 fe...",['1 NM N of HWD'],HWD,1,N,1.852,0,Hayward Exec,37.658929,-122.121737,KHWD


In [23]:
new_df['Distance'] = pd.to_numeric(new_df['Distance'],errors = 'coerce')
dist_kilo = []

for i in range(len(new_df)):
    distanceKilo = new_df['Distance'][i] * 1.852 # converting NM to kilometers
    dist_kilo.append(distanceKilo)

bearDegree = pd.DataFrame(new_df['Bearing'])
bearDegree = bearDegree.replace({'Bearing': bearing_deg})

new_df['Distance_Kilometers'] = dist_kilo
new_df['Bearing_Degrees'] = bearDegree
new_df['Airport_Latitude'] = uas_air_loc['LATITUDE']
new_df['Airport_Longitude'] = uas_air_loc['LONGITUDE']
new_df

,REMARKS,UAS_LOC,IDENT,Distance,Bearing,Distance_Kilometers,Bearing_Degrees,Airport_Latitude,Airport_Longitude
0,"PIC reported a white UAS SW bound at 8,000 fee...",['7 NM NW of DEN'],DEN,7,NW,12.964,315,39.861667,-104.673167
1,"PIC reported a white UAS while N bound at 10,2...",['9 NM NW of DEN'],DEN,9,NW,16.668,315,39.861667,-104.673167
2,PIC reported 2 UAS below on right side while S...,['7 NM N of LGA'],LGA,7,N,12.964,0,40.777250,-73.872611
3,Unreported call sign reported a UAS 5 NM E of ...,['5 NM E of BUR'],BUR,5,E,9.260,90,34.200694,-118.358667
4,"PIC reported a UAS while NNE bound at 1,000 fe...",['1 NM N of HWD'],HWD,1,N,1.852,0,37.658929,-122.121737


In [24]:
# Merge, drop and rename columns
new_df2 = pd.merge(new_df, airportsC, on='IDENT', how='left')
new_df2 = new_df2.drop(['Airport_Latitude', 'Airport_Longitude'], 1)
new_df2 = new_df2.rename({'LONGITUDE': 'Airport_Longitude', 'LATITUDE': 'Airport_Latitude'}, axis=1)
new_df2

,REMARKS,UAS_LOC,IDENT,Distance,Bearing,Distance_Kilometers,Bearing_Degrees,NAME,Airport_Latitude,Airport_Longitude,ICAO_ID
0,"PIC reported a white UAS SW bound at 8,000 fee...",['7 NM NW of DEN'],DEN,7,NW,12.964,315,Denver Intl,39.861667,-104.673167,KDEN
1,"PIC reported a white UAS while N bound at 10,2...",['9 NM NW of DEN'],DEN,9,NW,16.668,315,Denver Intl,39.861667,-104.673167,KDEN
2,PIC reported 2 UAS below on right side while S...,['7 NM N of LGA'],LGA,7,N,12.964,0,Laguardia,40.777250,-73.872611,KLGA
3,Unreported call sign reported a UAS 5 NM E of ...,['5 NM E of BUR'],BUR,5,E,9.260,90,Bob Hope,34.200694,-118.358667,KBUR
4,"PIC reported a UAS while NNE bound at 1,000 fe...",['1 NM N of HWD'],HWD,1,N,1.852,0,Hayward Exec,37.658929,-122.121737,KHWD


In [25]:
# Calculate UAS lat/long
uas_lat = []
uas_long = []

for i in range(len(new_df)):
    lat_airport = pd.to_numeric(new_df2['Airport_Latitude'][i])
    long_airport = pd.to_numeric(new_df2['Airport_Longitude'][i])
    b = pd.to_numeric(new_df['Bearing_Degrees'][i])
    d = pd.to_numeric(new_df['Distance_Kilometers'][i])
    
    origin = geopy.Point(lat_airport, long_airport)
    destination = geodesic(kilometers=d).destination(origin,b)
    
    lat2, lon2, = destination.latitude, destination.longitude
    
    uas_lat.append(lat2)
    uas_long.append(lon2)

# Append UAS Lat/Long information to DataFrame
new_df2['UAS_Latitude'] = uas_lat
new_df2['UAS_Longitude'] = uas_long
new_df2

,REMARKS,UAS_LOC,IDENT,Distance,Bearing,Distance_Kilometers,Bearing_Degrees,NAME,Airport_Latitude,Airport_Longitude,ICAO_ID,UAS_Latitude,UAS_Longitude
0,"PIC reported a white UAS SW bound at 8,000 fee...",['7 NM NW of DEN'],DEN,7,NW,12.964,315,Denver Intl,39.861667,-104.673167,KDEN,39.944178,-104.780428
1,"PIC reported a white UAS while N bound at 10,2...",['9 NM NW of DEN'],DEN,9,NW,16.668,315,Denver Intl,39.861667,-104.673167,KDEN,39.967734,-104.811122
2,PIC reported 2 UAS below on right side while S...,['7 NM N of LGA'],LGA,7,N,12.964,0,Laguardia,40.777250,-73.872611,KLGA,40.893989,-73.872611
3,Unreported call sign reported a UAS 5 NM E of ...,['5 NM E of BUR'],BUR,5,E,9.260,90,Bob Hope,34.200694,-118.358667,KBUR,34.200653,-118.258197
4,"PIC reported a UAS while NNE bound at 1,000 fe...",['1 NM N of HWD'],HWD,1,N,1.852,0,Hayward Exec,37.658929,-122.121737,KHWD,37.675615,-122.121737


In [26]:
# Export dataframe to csv file
new_df2 = new_df2.drop(['Distance','Bearing','Distance_Kilometers','Bearing_Degrees','ICAO_ID'], 1)
new_df2 = new_df2.rename({'Airport_Longitude': 'Origin_Longitude', 'Airport_Latitude': 'Origin_Latitude'}, axis=1)
new_df2 = pd.merge(new_df2, nonstd, on='REMARKS', how='left')
new_df2.to_csv('COMPLETED-Non-std-5Airports.csv', index = False)

In [ ]:
##############################################################################################################

In [23]:
# Save the record that reference a runway
runway = final_df1[final_df1['IDENT'] == 'RWY']
runway

,REMARKS,UAS_LOC,IDENT,NAME,LATITUDE,LONGITUDE,ICAO_ID
2,Pilot reported a 4 foot silver UAS orbiting at...,[5 NM S of RWY],RWY,NaN,NaN,NaN,NaN


In [24]:
# Merge with original dataset to have all columns
runway = pd.merge(runway, nonstd, on='REMARKS', how='left')
runway

,REMARKS,UAS_LOC,IDENT,NAME,LATITUDE,LONGITUDE,ICAO_ID,dateonly,timeonly,DATE,...,NEARESTAIRPORT.CEDAR,AIRSPACECLASS.CEDAR,ACLOCATION.CEDAR,ACALTITUDE.CEDAR,ACHEADING.CEDAR,RELATIVECLOCKPOSITION.CEDAR,UASLONG.CEDAR,UASLAT.CEDAR,SUMMARY.CEDAR,QAFINDINGS.CEDAR
0,Pilot reported a 4 foot silver UAS orbiting at...,[5 NM S of RWY],RWY,NaN,NaN,NaN,NaN,8/11/2018,0:34,2018-08-11T00:34Z,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
# Drop unnecessary columns
runway = runway.drop(['IDENT', 'NAME', 'LATITUDE', 'LONGITUDE', 'ICAO_ID'], axis = 1)
runway

,REMARKS,UAS_LOC,dateonly,timeonly,DATE,CALLSIGN,POD,PRIMARYCODE,SECONDARYCODES,REPORTINGFACILITY,...,NEARESTAIRPORT.CEDAR,AIRSPACECLASS.CEDAR,ACLOCATION.CEDAR,ACALTITUDE.CEDAR,ACHEADING.CEDAR,RELATIVECLOCKPOSITION.CEDAR,UASLONG.CEDAR,UASLAT.CEDAR,SUMMARY.CEDAR,QAFINDINGS.CEDAR
0,Pilot reported a 4 foot silver UAS orbiting at...,[5 NM S of RWY],8/11/2018,0:34,2018-08-11T00:34Z,R51263,DEN,UAS,NaN,DHN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
# Export to csv file
runway.to_csv('Non-std-1Runway.csv', index = False)